In [ ]:
import pandas as pd
import geopandas as gpd
import networkx as nx
from ast import literal_eval
from collections import defaultdict

In [ ]:
df = pd.read_csv('mi_all_subs_pseudo_cois.csv')

In [ ]:
precincts_df = gpd.read_file('MI')

In [ ]:
vtds_by_county = defaultdict(set)
for row in precincts_df.itertuples():
  vtds_by_county[getattr(row, 'CountyFips')].add(getattr(row, 'VTD2016_x'))

In [ ]:
districtr_data = []
for row in df['districtr_data']:
  try:
    districtr_data.append(literal_eval(row))
  except ValueError:
    pass

In [ ]:
for submission in districtr_data:
  if 'assignment' not in submission['plan']:
    continue
  assignment = submission['plan']['assignment']
  counties_to_districts = defaultdict(set)
  districts_to_vtds = defaultdict(set)
  for vtd, dist in assignment.items():
    county = vtd[1:4]
    if isinstance(dist, list):
      assert len(dist) == 1, f'{vtd} {dist}'
      dist = dist[0]
    counties_to_districts[county].add(dist)
    districts_to_vtds[dist].add(vtd[1:])
    
  whole_counties = set()
  for county, districts in counties_to_districts.items():
    if len(districts) == 1:
      district = next(iter(districts))
      if districts_to_vtds[district].issubset(vtds_by_county[county]):
        whole_counties.add(county)
  print(whole_counties)